In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py

import tensorflow as tf
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices(device_type="GPU")[0], True)

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

from msfm.grid_pipeline import GridPipeline
from msfm.utils import logger, input_output, files, scales, observation

# CosmoGrid

In [3]:
tfr_pattern = "/pscratch/sd/a/athomsen/v11desy3/v9/linear_bias/tfrecords/grid/DESy3_grid_dmb_????.tfrecord"

conf = "/global/homes/a/athomsen/multiprobe-simulation-forward-model/configs/v9/linear_bias.yaml"
conf = files.load_config(conf)
# conf = "/cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/configs/v8/linear_bias.yaml"
params = ["Om", "s8", "w0", "bg", "n_bg"]
filename_suffix = "_v9"

In [4]:
grid_pipe = GridPipeline(
    conf=conf,
    params=params,
    with_lensing=False,
    with_clustering=True,
    with_padding=False,
    apply_norm=False,
)

data_vec_pix = grid_pipe.data_vec_pix
n_side = 512
n_pix = hp.nside2npix(n_side)
n_z = grid_pipe.n_z_metacal + grid_pipe.n_z_maglim
n_z_metacal = grid_pipe.n_z_metacal
n_noise = 3

24-07-25 05:03:00     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_512.h5 
24-07-25 05:03:00     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_512.h5 


In [5]:
grid_dset = grid_pipe.get_dset(
    tfr_pattern=tfr_pattern,
    noise_indices=n_noise,
    local_batch_size=16,
    n_readers=1,
    n_prefetch=0,
    # is_eval=False,
    # examples_shuffle_buffer=512,
)

24-07-25 05:03:02 grid_pipelin INF   n_workers is not set, using tf.data.AUTOTUNE. This might produce unexpected RAM usage. 
24-07-25 05:03:02 grid_pipelin INF   drop_remainder is not set, using drop_remainder = False 
24-07-25 05:03:02 grid_pipelin INF   Including noise_indices = [0, 1, 2] 
24-07-25 05:03:02 grid_pipelin INF   Interleaving with n_readers = 1 
24-07-25 05:03:02 grid_pipelin INF   Batching into 16 elements locally 
24-07-25 05:03:03 grid_pipelin WAR   Tracing _augmentations 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) a

In [6]:
means = []
for dv, cosmo, index in grid_dset.take(10):
    print(cosmo[:,0])
    print(np.mean(dv, axis=1), "\n")
    
    means.append(np.mean(dv, axis=1))

tf.Tensor([0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3], shape=(16,), dtype=float32)
[[0.7774759  0.5419476  0.55053765 0.7340824 ]
 [0.7792002  0.54380727 0.5489048  0.7343787 ]
 [0.7763865  0.54344505 0.54976636 0.7363603 ]
 [0.75780725 0.53792286 0.5488085  0.73709846]
 [0.7571893  0.5372879  0.5478386  0.7375952 ]
 [0.7568432  0.53713274 0.5474794  0.7381962 ]
 [0.75951916 0.5417993  0.5468946  0.7374273 ]
 [0.7594288  0.5434929  0.54607946 0.740497  ]
 [0.7602783  0.5408388  0.54748726 0.7387613 ]
 [0.76565856 0.5378951  0.5544641  0.7312693 ]
 [0.7645675  0.5385429  0.5544382  0.7319116 ]
 [0.76365036 0.5387348  0.55611974 0.73128045]
 [0.75968754 0.54145414 0.5491709  0.73657024]
 [0.75782347 0.5404033  0.54687697 0.7372213 ]
 [0.7598362  0.54003006 0.54875714 0.7352812 ]
 [0.76008177 0.53635424 0.5502934  0.7417371 ]] 

tf.Tensor([0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3], shape=(16,), dtype=float32)
[[0.7599728  0.53542054 0.5512986  

In [7]:
# for i, (dv, cosmo, index) in enumerate(grid_dset.take(1000)):
#     if i > 900:
#         print(cosmo[:,0])
#         print(np.mean(dv, axis=1), "\n")

# External Mock

In [13]:
obs_file = "/global/u2/a/athomsen/multiprobe-simulation-forward-model/data/mock_observations/DESY3_Buzzard_mock.h5"
obs_label = f"Buzzard"

# obs_file = "/global/u2/a/athomsen/multiprobe-simulation-forward-model/data/mock_observations/DESY3_Cardinal_mock.h5"
# obs_label = f"Cardinal"

with h5py.File(obs_file, "r") as f:
    gc_map = []
    for i in range(1,5):
        gc_map.append(f[f"maglim/galaxy_counts_bin{i}"][:])
    gc_map = np.stack(gc_map, axis=-1)
    
obs_dv, obs_cl = observation.forward_model_observation_map(
    wl_gamma_map=np.zeros((hp.nside2npix(512),4,2)),
    gc_count_map=gc_map,
    conf=conf,
    apply_norm=True,
    with_padding=False,
    nest=False,
)

24-07-25 05:04:59     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_512.h5 
24-07-25 05:04:59     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_v11_512.h5 


In [14]:
np.mean(obs_dv[:,4:], axis=0)

array([0.60962147, 0.4304412 , 0.44149333, 0.59470934], dtype=float32)

In [ ]:
np.mean(gc_map, axis=0)

In [16]:
# CosmoGrid / buzzard
np.mean(means, axis=(0,1))/np.mean(obs_dv[:,4:], axis=0)

array([1.2445192, 1.2530206, 1.2427323, 1.2377559], dtype=float32)

In [18]:
1.25**2

1.5625